In [1]:
import os
import sys
import git
import pathlib

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

print(f"Project Root Directory: {PROJ_ROOT}")

Project Root Directory: /repos/drl_csense


In [4]:
import gymnasium as gym

In [5]:
from lib.env_utils import AtariWrapper_NoisyWarp, make_atari_env_Custom_VecFrameStack

In [7]:
import imageio
import ipyplot

In [ ]:
def generate_random_frames(trial_env, duration=10, fps=120):
    no_of_frames = duration*fps*2 #only every other frame is used for animation
    
    images = []
    obss = []
    obs = trial_env.reset()
    img = trial_env.render(mode="rgb_array")
    for i in range(no_of_frames):
        images.append(img)
        obss.append(obs[0,:,:,-1])
        action = trial_env.action_space.sample()
        obs, reward, done, info = trial_env.step([action])
        img = trial_env.render(mode="rgb_array")
    return obss, images

In [32]:
ENV_LIST = ["BreakoutNoFrameskip-v4", "BankHeistNoFrameskip-v4", "WizardOfWorNoFrameskip-v4"]



# env_id = "BreakoutNoFrameskip-v4"
# noise = 0.4
NOISE_LIST = [0.0, 0.1, 0.2, 0.3, 0.4]
for env_id in ENV_LIST:
    for noise in NOISE_LIST:
        wrapper = AtariWrapper_NoisyWarp
        wrapper_kwargs = {"noise":noise}
        
        trial_env = make_atari_env_Custom_VecFrameStack(env_id=env_id,
                                                        n_envs=1,
                                                        monitor_dir=None,
                                                        seed=161803,
                                                        wrapper_class=wrapper,
                                                        wrapper_kwargs=wrapper_kwargs
                                                         )
        
        
        
        # Generate frames
        duration = 5
        obss, images = generate_random_frames(trial_env, duration=duration, fps=200) # generates fps*duration*2 frames
            
        
        # Convert frames to animation
        img_gif_file = f"./gifs/{env_id}--noise_{noise}--img.gif"
        imageio.mimsave(img_gif_file, 
                        [np.array(img) for i, img in enumerate(images) if i%2 == 0], duration=duration)
        
        # Convert obss to animation
        obs_gif_file = f"./gifs/{env_id}--noise_{noise}--obs.gif"
        imageio.mimsave(obs_gif_file, 
                        [np.array(obs) for i, obs in enumerate(obss) if i%2 == 0], duration=duration)
        
        trial_env.close()

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:364: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
